# Chatbot using NLP and Neural Networks in Python

Tag means classes

Patterns means what user is going to ask

Response is chatbot response

In [ ]:
data={"intents":[
    {"tag":"greeting",
     "patterns":["Hello","How are you?","Hi There","Hi","What's up"],
     "responses":["Howdy Partner!","Hello","How are you doing?","Greetings!","How do you do"]
        },
    {"tag":"age",
     "patterns":["how old are you","when is your birthday","when was you born"],
     "responses":["I am 24 years old","I was born in 1966","My birthday is July 3rd and I was born in 1996","03/07/1996"]
        },
    {"tag":"date",
     "patterns":["what are you doing this weekend",
                "do you want to hangout sometime?","what are your plans for this week"],
     "responses":["I am available this week","I don't have any plans","I am not busy"]
        },
    {"tag":"name",
     "patterns":["what's your name","what are you called","who are you"],
     "responses":["My name is Kippi","I'm Kippi","Kippi"]
        },
    {"tag":"goodbye",
     "patterns":["bye","g2g","see ya","adios","cya"],
     "responses":["It was nice speaking to you","See you later","Speak Soon"]
        },
]}

For each tag we created, we would specify patterns. Essentially this defines the different ways of how a user may pose a query to the chatbot.

The chatbot would then take these patterns and use them as training data to determine what someone is asking and the chatbot response would be relevant to that qustion.

In [ ]:
import json
import string
import random

import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout

nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package punkt to C:\Users\Mohit
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Mohit
[nltk_data]     Tripathi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In order to create our training data below steps to be followed

Create a vocabulary of all the words used in the patterns

Create a list of the classes- tags of each intent

Create a list of all the patterns within the intents file

Create a list of all the associated tags to go with each patterns in the intents file.

Intialise lemmatizer to get stem of words

In [ ]:
lemmatizer=WordNetLemmatizer()

words=[]
classes=[]
doc_x=[]
doc_y=[]

Loop through all the intents

Tokenize each pattern and append token to words, the patterns and the associated tag to their associated list

In [ ]:
for intent in data["intents"]:
    for pattern in intent["patterns"]:
        tokens=nltk.word_tokenize(pattern)
        words.extend(tokens)
        doc_x.append(pattern)
        doc_y.append(intent["tag"])
    if intent["tag"] not in classes:
        classes.append(intent["tag"])

Lemmatize all the words in the vocab and convert them to lowercase

In [ ]:
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]

Sorting the vocab and classes in alphabeical order and taking the set to ensure no duplicates occur

In [ ]:
words=sorted(set(words))
classes=sorted(set(classes))

In [ ]:
print(words)

["'s", 'adios', 'are', 'birthday', 'born', 'bye', 'called', 'cya', 'do', 'doing', 'for', 'g2g', 'hangout', 'hello', 'hi', 'how', 'is', 'name', 'old', 'plan', 'see', 'sometime', 'there', 'this', 'to', 'up', 'wa', 'want', 'week', 'weekend', 'what', 'when', 'who', 'ya', 'you', 'your']


In [ ]:
print(classes)

['age', 'date', 'goodbye', 'greeting', 'name']


In [ ]:
print(doc_x)

['Hello', 'How are you?', 'Hi There', 'Hi', "What's up", 'how old are you', 'when is your birthday', 'when was you born', 'what are you doing this weekend', 'do you want to hangout sometime?', 'what are your plans for this week', "what's your name", 'what are you called', 'who are you', 'bye', 'g2g', 'see ya', 'adios', 'cya']


In [ ]:
print(doc_y)

['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'age', 'age', 'age', 'date', 'date', 'date', 'name', 'name', 'name', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye']


Training Data

In [ ]:
training=[]
out_empty=[0]*len(classes)

# creating a bag of words model

for idx, doc in enumerate(doc_x):
    bow=[]
    text=lemmatizer.lemmatize(doc.lower())
    for word in words:
        bow.append(1) if word in text else bow.append(0)
    output_row=list(out_empty)
    output_row[classes.index(doc_y[idx])]=1

    training.append([bow, output_row])

random.shuffle(training)

training=np.array(training,dtype=object)

train_X=np.array(list(training[:,0]))
train_y=np.array(list(training[:,1]))

The model will look at the features and predict the tag asscoiated with the features and then will select an appropiate message/response from the tag.

In [ ]:
input_shape=(len(train_X[0]),)
output_shape=len(train_y[0])

epochs=500

In [ ]:
model=Sequential()
model.add(Dense(128, input_shape=input_shape,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(output_shape,activation='softmax'))
adam=tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)

model.compile(loss='categorical_crossentropy',
             optimizer="adam",
             metrics="accuracy")

print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               4736      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 5)                 325       
                                                                 
Total params: 13,317
Trainable params: 13,317
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(x=train_X, y=train_y, epochs=500, verbose=1)

Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 1.5180 - accuracy: 0.3158
Epoch 2/500
1/1 [==============================] - 0s 10ms/step - loss: 1.5512 - accuracy: 0.3158
Epoch 3/500
1/1 [==============================] - 0s 8ms/step - loss: 1.6077 - accuracy: 0.1579
Epoch 4/500
1/1 [==============================] - 0s 12ms/step - loss: 1.4972 - accuracy: 0.3158
Epoch 5/500
1/1 [==============================] - 0s 9ms/step - loss: 1.5012 - accuracy: 0.2105
Epoch 6/500
1/1 [==============================] - 0s 18ms/step - loss: 1.4813 - accuracy: 0.4737
Epoch 7/500
1/1 [==============================] - 0s 12ms/step - loss: 1.4511 - accuracy: 0.4211
Epoch 8/500
1/1 [==============================] - 0s 12ms/step - loss: 1.4776 - accuracy: 0.4737
Epoch 9/500
1/1 [==============================] - 0s 14ms/step - loss: 1.5922 - accuracy: 0.3158
Epoch 10/500
1/1 [==============================] - 0s 12ms/step - loss: 1.4441 - accuracy: 0.4211
Epoch 11/500
1/1 [=====

1/1 [==============================] - 0s 16ms/step - loss: 0.0487 - accuracy: 1.0000
Epoch 166/500
1/1 [==============================] - 0s 17ms/step - loss: 0.0909 - accuracy: 1.0000
Epoch 167/500
1/1 [==============================] - 0s 14ms/step - loss: 0.0566 - accuracy: 1.0000
Epoch 168/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0486 - accuracy: 1.0000
Epoch 169/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0425 - accuracy: 1.0000
Epoch 170/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0919 - accuracy: 1.0000
Epoch 171/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0550 - accuracy: 1.0000
Epoch 172/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0570 - accuracy: 1.0000
Epoch 173/500
1/1 [==============================] - 0s 15ms/step - loss: 0.1214 - accuracy: 1.0000
Epoch 174/500
1/1 [==============================] - 0s 17ms/step - loss: 0.0678 - accuracy: 1.0000
Epoch 175/500


1/1 [==============================] - 0s 11ms/step - loss: 0.0142 - accuracy: 1.0000
Epoch 248/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0283 - accuracy: 1.0000
Epoch 249/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 250/500
1/1 [==============================] - 0s 14ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 251/500
1/1 [==============================] - 0s 21ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 252/500
1/1 [==============================] - 0s 19ms/step - loss: 0.0275 - accuracy: 1.0000
Epoch 253/500
1/1 [==============================] - 0s 17ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 254/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0159 - accuracy: 1.0000
Epoch 255/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0405 - accuracy: 1.0000
Epoch 256/500
1/1 [==============================] - 0s 10ms/step - loss: 0.0170 - accuracy: 1.0000
Epoch 257/500


1/1 [==============================] - 0s 13ms/step - loss: 0.0121 - accuracy: 1.0000
Epoch 330/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0310 - accuracy: 1.0000
Epoch 331/500
1/1 [==============================] - 0s 14ms/step - loss: 0.0585 - accuracy: 1.0000
Epoch 332/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 333/500
1/1 [==============================] - 0s 18ms/step - loss: 0.0242 - accuracy: 1.0000
Epoch 334/500
1/1 [==============================] - 0s 17ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 335/500
1/1 [==============================] - 0s 14ms/step - loss: 0.0231 - accuracy: 1.0000
Epoch 336/500
1/1 [==============================] - 0s 14ms/step - loss: 0.0271 - accuracy: 1.0000
Epoch 337/500
1/1 [==============================] - 0s 12ms/step - loss: 0.0183 - accuracy: 1.0000
Epoch 338/500
1/1 [==============================] - 0s 15ms/step - loss: 0.0076 - accuracy: 1.0000
Epoch 339/500


1/1 [==============================] - 0s 27ms/step - loss: 0.0929 - accuracy: 0.9474
Epoch 412/500
1/1 [==============================] - 0s 20ms/step - loss: 0.0074 - accuracy: 1.0000
Epoch 413/500
1/1 [==============================] - 0s 31ms/step - loss: 0.0082 - accuracy: 1.0000
Epoch 414/500
1/1 [==============================] - 0s 28ms/step - loss: 0.0091 - accuracy: 1.0000
Epoch 415/500
1/1 [==============================] - 0s 30ms/step - loss: 0.0050 - accuracy: 1.0000
Epoch 416/500
1/1 [==============================] - 0s 21ms/step - loss: 0.0097 - accuracy: 1.0000
Epoch 417/500
1/1 [==============================] - 0s 22ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 418/500
1/1 [==============================] - 0s 21ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 419/500
1/1 [==============================] - 0s 26ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 420/500
1/1 [==============================] - 0s 25ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 421/500


1/1 [==============================] - 0s 14ms/step - loss: 0.0292 - accuracy: 1.0000
Epoch 494/500
1/1 [==============================] - 0s 16ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 495/500
1/1 [==============================] - 0s 17ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 496/500
1/1 [==============================] - 0s 25ms/step - loss: 0.0017 - accuracy: 1.0000
Epoch 497/500
1/1 [==============================] - 0s 25ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 498/500
1/1 [==============================] - 0s 16ms/step - loss: 9.8628e-04 - accuracy: 1.0000
Epoch 499/500
1/1 [==============================] - 0s 29ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 500/500
1/1 [==============================] - 0s 21ms/step - loss: 0.0048 - accuracy: 1.0000


In [ ]:
def clean_text(text):
    tokens=nltk.word_tokenize(text)
    tokens=[lemmatizer.lemmatize(word) for word in tokens]
    return tokens


def bag_of_words(text,vocab):
    tokens=clean_text(text)
    bow=[0]*len(vocab)
    for w in tokens:
        for idx, word in enumerate(vocab):
            if word==w:
                bow[idx]=1
    return np.array(bow)

In [ ]:
def pred_class(text, vocab,labels):
    bow=bag_of_words(text, vocab)
    result=model.predict(np.array([bow]))[0]
    thresh=0.2
    y_pred=[[idx,res] for idx, res in enumerate(result) if res>thresh]

    y_pred.sort(key=lambda x:x[1], reverse=True)
    return_list=[]
    for r in y_pred:
        return_list.append(labels[r[0]])
    return return_list

def get_response(intents_list, intents_json):
    tag=intents_list[0]
    list_of_intents=intents_json["intents"]
    for i in list_of_intents:
        if i["tag"]==tag:
            result=random.choice(i["responses"])
            break
    return result

Running the chatbot

In [ ]:
while True:
    message=input("")
    intents=pred_class(message, words, classes)
    result=get_response(intents,data)
    print(result)

hello
1/1 [==============================] - 0s 366ms/step
Greetings!
what is your name
1/1 [==============================] - 0s 25ms/step
I'm Kippi
goodbye
1/1 [==============================] - 0s 22ms/step
See you later


KeyboardInterrupt: Interrupted by user